In [42]:
%run data.ipynb
%run models.ipynb
%run binarizers.ipynb

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [38]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)

In [39]:
epochs = 100
lr = 1e-3
rho = 0.5
optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [41]:
def train(epochs):
    net.train()
    
    for epoch in range(epochs):
        for idx, (data, target) in enumerate(train_loader):
            if device == 'cuda':
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)

            optimizer.zero_grad()
            output = net(data)
            loss = criterion(output, target)

            if epoch % 40 == 0:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']*0.1

            loss.backward()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.data.copy_(p.org)
            
            optimizer.step()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.org.copy_(p.data.clamp_(-1,1))
            
            
            if idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, idx * len(data), len(train_loader.dataset),
                    100. * idx / len(train_loader), loss.item()))
        test()

def test():
    net.eval()
    test_loss, correct = 0, 0
    
    for data, target in test_loader:
        if device == 'cuda':
            data, target = data.cuda(), target.cuda()
            
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
        
        output = net(data)
        
        test_loss += criterion(output, target).item()
        pred = output.data.max(dim=1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

train(epochs)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 1.648251
Train Epoch: 0 [2560/60000 (4%)]	Loss: 1.609330
Train Epoch: 0 [5120/60000 (9%)]	Loss: 1.680063
Train Epoch: 0 [7680/60000 (13%)]	Loss: 1.629855
Train Epoch: 0 [10240/60000 (17%)]	Loss: 1.678252
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.643955
Train Epoch: 0 [15360/60000 (26%)]	Loss: 1.645762
Train Epoch: 0 [17920/60000 (30%)]	Loss: 1.679273
Train Epoch: 0 [20480/60000 (34%)]	Loss: 1.655588
Train Epoch: 0 [23040/60000 (38%)]	Loss: 1.593444
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.677775
Train Epoch: 0 [28160/60000 (47%)]	Loss: 1.632152
Train Epoch: 0 [30720/60000 (51%)]	Loss: 1.634738
Train Epoch: 0 [33280/60000 (55%)]	Loss: 1.634408
Train Epoch: 0 [35840/60000 (60%)]	Loss: 1.683200
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.634437
Train Epoch: 0 [40960/60000 (68%)]	Loss: 1.638308
Train Epoch: 0 [43520/60000 (72%)]	Loss: 1.682880
Train Epoch: 0 [46080/60000 (77%)]	Loss: 1.664504
Train Epoch: 0 [48640/60000 (81%)]	Loss: 1.631248
Train Epoc

KeyboardInterrupt: 